In [1]:
#sys path
from sys import path
from pathlib import Path

module_path = str(Path.cwd().parents[0])

if module_path not in path:
    path.append(module_path)
    
path.append(module_path + '\\functions')


# libraries
import pandas as pd
import numpy as np

import scipy.linalg
import matplotlib.pyplot as plt

from gp_regression import GPR
from gp_ccopf_hybrid import GP_CCOPF
import save_outputs
import linear_model
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from sklearn.metrics import mean_squared_error

# Dataset

In [2]:
# Data Set AC

df_ac = pd.read_csv('datasets\dataset_pandapower_AC.csv')
df_ac = df_ac.drop(['Unnamed: 0'], axis=1)
df_ac[['U3','U4','U5', 'U6', 'U7', 'U8']] = df_ac[['U3','U4','U5', 'U6', 'U7', 'U8']]*345
df_ac.head()

,Pg0,Pg1,Pg2,Load_P1,Load_P2,Load_P3,Load_Q1,Load_Q2,Load_Q3,RS_P1,RS_P2,RS_Q1,RS_Q2,P0,P1,P2,P3,P4,P5,P6,P7,P8,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,U0,U1,U2,U3,U4,U5,U6,U7,U8,Ud0,Ud1,Ud2,Ud3,Ud4,Ud5,Ud6,Ud7,Ud8,P_03,P_34,P_45,P_25,P_56,P_67,P_71,P_78,P_83
0,59.133076,91.046121,58.858774,77.773848,82.096546,109.642025,25.924616,28.733791,43.856810,33.596406,29.744812,10.078922,9.914937,-58.856484,-91.046121,-58.858774,-33.596406,77.773848,-29.744812,82.096546,0.0,109.642025,2.350474,6.317373,23.605955,-10.078922,25.924616,-9.914937,28.733791,0.0,43.856810,1.0,1.0,1.0,345.665016,343.252822,349.974771,345.013234,346.918104,338.032810,0.0,3.157380,2.631294,-1.939037,-3.667944,0.682807,-1.691738,-0.086676,-4.767008,58.856484,33.060233,-44.899384,58.858774,42.909810,-39.411861,-91.046121,51.501723,-58.999419
1,66.599881,157.267682,92.238488,102.301461,104.046287,142.384024,34.100487,36.416200,56.953609,19.325453,17.635380,5.797636,5.878460,-68.289225,-157.267682,-92.238488,-19.325453,102.301461,-17.635380,104.046287,0.0,142.384024,-28.625599,-17.542616,4.402647,-5.797636,34.100487,-5.878460,36.416200,0.0,56.953609,1.0,1.0,1.0,339.582781,334.185957,346.392396,339.400861,342.898296,327.189586,0.0,9.338553,6.098185,-2.290268,-3.989830,3.012205,0.894783,3.663024,-4.928414,68.289225,32.793857,-69.718053,92.238488,38.135506,-66.114022,-157.267682,90.768012,-54.374524
2,79.074452,134.389013,64.829596,95.968023,108.666208,136.360049,31.989341,38.033173,54.544020,38.277040,28.240721,11.483112,9.413574,-79.420587,-134.389013,-64.829596,-38.277040,95.968023,-28.240721,108.666208,0.0,136.360049,-14.588350,-10.672416,11.381477,-11.483112,31.989341,-9.413574,38.033173,0.0,54.544020,1.0,1.0,1.0,342.464863,337.888287,347.548216,340.160059,343.921711,331.109711,0.0,5.010390,1.913795,-2.641406,-5.103047,-0.247433,-2.591240,0.177119,-6.083925,79.420587,46.620172,-49.730510,64.829596,42.331427,-66.579714,-134.389013,67.419083,-70.460191
3,49.510102,110.728146,72.459392,73.641679,89.800681,112.311358,24.547226,31.430238,44.924543,23.375186,22.872133,7.012556,7.624044,-49.481764,-110.728146,-72.459392,-23.375186,73.641679,-22.872133,89.800681,0.0,112.311358,-2.964563,1.712063,19.577410,-7.012556,24.547226,-7.624044,31.430238,0.0,44.924543,1.0,1.0,1.0,344.551221,342.571270,349.265312,343.704630,346.193459,336.377427,0.0,5.757109,4.653073,-1.635364,-2.799237,2.249231,-0.125910,1.802480,-4.028748,49.481764,22.324345,-51.403202,72.459392,42.880168,-47.150686,-110.728146,63.385933,-50.224048
4,74.088754,104.415714,69.420999,86.981422,87.469644,129.760427,28.993807,30.614375,51.904171,28.081316,31.604788,8.424395,10.534929,-74.322851,-104.415714,-69.420999,-28.081316,86.981422,-31.604788,87.469644,0.0,129.760427,-10.644654,-1.099562,18.777980,-8.424395,28.993807,-10.534929,30.614375,0.0,51.904171,1.0,1.0,1.0,343.202637,339.977021,349.078597,343.374202,345.497281,333.129085,0.0,3.763322,3.242836,-2.466437,-4.250579,0.938615,-1.657588,0.026943,-5.770507,74.322851,33.978340,-53.204661,69.420999,46.684088,-41.055929,-104.415714,63.214346,-67.873473


In [3]:
# Data Set DC

df_dc = pd.read_csv('datasets\dataset_pandapower_DC.csv')
df_dc = df_dc.drop(['Unnamed: 0'], axis=1)
df_dc[['U3','U4','U5', 'U6', 'U7', 'U8']] = df_dc[['U3','U4','U5', 'U6', 'U7', 'U8']]*345
df_dc.head()

,Pg0,Pg1,Pg2,Load_P1,Load_P2,Load_P3,Load_Q1,Load_Q2,Load_Q3,RS_P1,RS_P2,RS_Q1,RS_Q2,P0,P1,P2,P3,P4,P5,P6,P7,P8,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,U0,U1,U2,U3,U4,U5,U6,U7,U8,Ud0,Ud1,Ud2,Ud3,Ud4,Ud5,Ud6,Ud7,Ud8,P_03,P_34,P_45,P_25,P_56,P_67,P_71,P_78,P_83
0,59.133076,91.046121,58.858774,77.773848,82.096546,109.642025,25.924616,28.733791,43.856810,33.596406,29.744812,10.078922,9.914937,-56.266306,-91.046121,-58.858774,-33.596406,77.773848,-29.744812,82.096546,0.0,109.642025,18.650177,-14.451768,16.412049,-15.0,30.0,-10.0,35.0,0.0,50.0,1.0,1.04848,1.033828,348.93565,347.740245,360.15419,355.51357,359.375274,343.024226,0.0,3.347258,2.906565,-1.856921,-3.538242,0.930363,-1.537242,0.086909,-4.679968,56.266306,31.896281,-45.877568,58.858774,42.726019,-39.370528,-91.046121,51.675594,-57.966431
1,66.599881,157.267682,92.238488,102.301461,104.046287,142.384024,34.100487,36.416200,56.953609,19.325453,17.635380,5.797636,5.878460,-62.264768,-157.267682,-92.238488,-19.325453,102.301461,-17.635380,104.046287,0.0,142.384024,18.650177,-14.451768,16.412049,-15.0,30.0,-10.0,35.0,0.0,50.0,1.0,1.04848,1.033828,348.93565,347.740245,360.15419,355.51357,359.375274,343.024226,0.0,9.501295,6.437472,-2.054885,-3.659278,3.340534,1.145340,3.869561,-4.546124,62.264768,30.436891,-71.864570,92.238488,38.009299,-66.036988,-157.267682,91.230694,-51.153330
2,79.074452,134.389013,64.829596,95.968023,108.666208,136.360049,31.989341,38.033173,54.544020,38.277040,28.240721,11.483112,9.413574,-75.257911,-134.389013,-64.829596,-38.277040,95.968023,-28.240721,108.666208,0.0,136.360049,18.650177,-14.451768,16.412049,-15.0,30.0,-10.0,35.0,0.0,50.0,1.0,1.04848,1.033828,348.93565,347.740245,360.15419,355.51357,359.375274,343.024226,0.0,5.243250,2.290026,-2.483689,-4.854100,0.113351,-2.316440,0.430798,-5.822949,75.257911,44.968985,-50.999039,64.829596,42.071278,-66.594930,-134.389013,67.794083,-68.565966
3,49.510102,110.728146,72.459392,73.641679,89.800681,112.311358,24.547226,31.430238,44.924543,23.375186,22.872133,7.012556,7.624044,-46.318862,-110.728146,-72.459392,-23.375186,73.641679,-22.872133,89.800681,0.0,112.311358,18.650177,-14.451768,16.412049,-15.0,30.0,-10.0,35.0,0.0,50.0,1.0,1.04848,1.033828,348.93565,347.740245,360.15419,355.51357,359.375274,343.024226,0.0,5.934039,4.924150,-1.528632,-2.635783,2.491302,0.025572,1.968879,-3.899920,46.318862,21.003726,-52.637954,72.459392,42.693571,-47.107110,-110.728146,63.621036,-48.690322
4,74.088754,104.415714,69.420999,86.981422,87.469644,129.760427,28.993807,30.614375,51.904171,28.081316,31.604788,8.424395,10.534929,-70.688677,-104.415714,-69.420999,-28.081316,86.981422,-31.604788,87.469644,0.0,129.760427,18.650177,-14.451768,16.412049,-15.0,30.0,-10.0,35.0,0.0,50.0,1.0,1.04848,1.033828,348.93565,347.740245,360.15419,355.51357,359.375274,343.024226,0.0,4.022646,3.605159,-2.332894,-4.041220,1.274326,-1.408525,0.283534,-5.564787,70.688677,32.408607,-54.572816,69.420999,46.452971,-41.016673,-104.415714,63.399041,-66.361386


In [4]:
df = df_ac - df_dc
df.head()

,Pg0,Pg1,Pg2,Load_P1,Load_P2,Load_P3,Load_Q1,Load_Q2,Load_Q3,RS_P1,RS_P2,RS_Q1,RS_Q2,P0,P1,P2,P3,P4,P5,P6,P7,P8,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,U0,U1,U2,U3,U4,U5,U6,U7,U8,Ud0,Ud1,Ud2,Ud3,Ud4,Ud5,Ud6,Ud7,Ud8,P_03,P_34,P_45,P_25,P_56,P_67,P_71,P_78,P_83
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.590178,4.263256e-14,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,-16.299704,20.769141,7.193906,4.921078,-4.075384,0.085063,-6.266209,0.0,-6.143190,-1.000000e-08,-0.04848,-0.033828,-3.270635,-4.487422,-10.179419,-10.500336,-12.457171,-4.991416,0.0,-0.189878,-0.275271,-0.082115,-0.129702,-0.247556,-0.154496,-0.173585,-0.087040,2.590178,1.163952,0.978183,-7.034373e-12,0.183791,-0.041333,8.839152e-12,-0.173871,-1.032988
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.024457,2.842171e-14,5.684342e-14,0.0,0.0,0.0,0.0,0.0,0.0,-47.275777,-3.090848,-12.009402,9.202364,4.100487,4.121540,1.416200,0.0,6.953609,-1.000000e-08,-0.04848,-0.033828,-9.352870,-13.554287,-13.761794,-16.112709,-16.476978,-15.834640,0.0,-0.162742,-0.339286,-0.235384,-0.330552,-0.328329,-0.250557,-0.206537,-0.382290,6.024457,2.356966,2.146517,-1.421085e-14,0.126208,-0.077034,-5.684342e-14,-0.462682,-3.221194
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.162676,-2.842171e-14,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,-33.238527,3.779353,-5.030573,3.516888,1.989341,0.586426,3.033173,0.0,4.544020,-1.000000e-08,-0.04848,-0.033828,-6.470787,-9.851957,-12.605974,-15.353511,-15.453563,-11.914515,0.0,-0.232861,-0.376230,-0.157716,-0.248947,-0.360783,-0.274800,-0.253679,-0.260976,4.162676,1.651188,1.268529,-1.421085e-14,0.260149,0.015216,0.000000e+00,-0.374999,-1.894224
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.162903,2.842171e-14,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,-21.614741,16.163831,3.165361,7.987444,-5.452774,2.375956,-3.569762,0.0,-5.075457,-1.000000e-08,-0.04848,-0.033828,-4.384429,-5.168975,-10.888879,-11.808941,-13.181815,-6.646799,0.0,-0.176930,-0.271077,-0.106732,-0.163454,-0.242072,-0.151482,-0.166400,-0.128828,3.162903,1.320619,1.234752,-9.862333e-12,0.186597,-0.043576,2.086153e-11,-0.235102,-1.533726
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.634174,0.000000e+00,-2.842171e-14,0.0,0.0,0.0,0.0,0.0,0.0,-29.294832,13.352206,2.365931,6.575605,-1.006193,-0.534929,-4.385625,0.0,1.904171,-1.000000e-08,-0.04848,-0.033828,-5.733014,-7.763224,-11.075594,-12.139368,-13.877993,-9.895141,0.0,-0.259325,-0.362323,-0.133544,-0.209359,-0.335711,-0.249063,-0.256590,-0.205720,3.634174,1.569733,1.368155,-1.421085e-14,0.231118,-0.039256,2.842171e-14,-0.184696,-1.512087


In [5]:
def _get_x(df):
    X = df[['Pg0', 'Pg1', 'Pg2', 'Load_P1', 'Load_P2', 'Load_P3', 'RS_P1', 'RS_P2']]
    return X 


def _get_y(df):
    y = df[['U3', 'U4', 'U5', 'U6', 'U7', 'U8', 
            'Q0', 'Q1', 'Q2', 
            'P_34', 'P_45', 'P_56', 'P_67', 'P_78', 'P_83']]
    return y 


X = _get_x(df_ac)
Y = _get_y(df)

In [6]:
Nu = 3
Nd = 5

n_train = 75
n_test = n_train + 25

X_train = X.iloc[:n_train]  
Y_train = Y.iloc[:n_train]
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()


X_test = X.iloc[n_train:n_test]
Y_test = Y.iloc[n_train:n_test]


X_d = X.iloc[n_train:n_test, Nu:Nu+Nd]

X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()
X_d = X_d.to_numpy()

# GP

In [7]:
model = 0


print('*****************')
print('GP')
print('*****************')

M=10
Nc=1000

#Train
if model==0:
    gp = GPR(X_train, Y_train, M, Nc, normalize=True)
    gp.save_model('models/IEEE9_hybrid')

#Load    
elif model==1:
    gp = GPR.load_model('models/IEEE9_hybrid')


print('*****************')
print('Hyper_parameters')
print('*****************')
gp.print_hyper_parameters()

*****************
GP
*****************

________________________________________
# Optimizing hyperparameters (N=75)
----------------------------------------
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 36007904873.6701
            Iterations: 2
            Function evaluations: 31
            Gradient evaluations: 2
* Output 1:  0.031248 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 436990955717.1039
            Iterations: 1
            Function evaluations: 11
            Gradient evaluations: 1
* Output 2:  0.023305 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 437509663773.99744
            Iterations: 1
            Function evaluations: 11
            Gradient evaluations: 1
* Output 3:  0.015623 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 427231678906.2959
            Iterations: 1
            Functio

In [8]:
print('*****************')
print('Validation - Validation Data')
print('*****************')


RMSE = gp.validate(X_test, Y_test, case='IEEE9')
print('RMSE:', np.mean(RMSE))
print('RMSE_u:', np.mean(RMSE[:6]))
print('RMSE_q:', np.mean(RMSE[6:9]))
print('RMSE_p:', np.mean(RMSE[9:15]))

*****************
Validation - Validation Data
*****************

________________________________________
# Validation of GP model 
----------------------------------------
* Num training samples: 75
* Num test samples: 25
----------------------------------------
* Root Mean Squared Error: 
	- State 1: 0.003921
	- State 2: 0.005187
	- State 3: 0.002232
	- State 4: 0.004404
	- State 5: 0.002791
	- State 6: 0.008009
	- State 7: 0.076681
	- State 8: 0.051926
	- State 9: 0.044927
	- State 10: 0.002563
	- State 11: 0.002818
	- State 12: 0.000706
	- State 13: 0.000454
	- State 14: 0.000753
	- State 15: 0.003352
----------------------------------------
RMSE: 0.014048170471758948
RMSE_u: 0.004423965860790684
RMSE_q: 0.05784475641000422
RMSE_p: 0.0017740821136045702


# GP CCOPF hybrid

### Linear Model Coefficients

In [9]:
save = 1

if save==0:
    dc = pd.read_csv('datasets\dataset_pandapower_DC.csv')
    A_coef, b_inter = linear_model.linear_model_IEEE9(dc)
    
    IEEE_dict = save_outputs.to_dict_coef(A_coef, b_inter)
    save_outputs.save_model(IEEE_dict, 'outputs\linear_coeff')
    
elif save==1:
    
    coeff_dict = save_outputs.load_model('outputs\linear_coeff')
    
    A_coef = np.array(coeff_dict['A'])
    b_inter = np.array(coeff_dict['b'])

A_coef.shape, b_inter.shape

((6, 8), (6,))

In [10]:
# Input mean
Xd = np.array([[ 90.,  100., 125., 50, 30]])

# Participation factors
alpha = np.array([[0.313, 0.405, 0.281]]).T

# Costs
R = np.eye(Nu) * [0.1100, 0.0850, 0.1225]
R2 = np.ones(Nu) * [5, 1.2, 1]

# Control boundaries
ulb = [ 10,  10,  10]
uub = [250, 300, 270]

# Output boundaries
xlb = [310.5, 310.5, 310.5, 310.5, 310.5, 310.5, -100, -100, -100, -200,  -200, -200, -200, -200, -200]
xub = [379.5, 379.5, 379.5, 379.5, 379.5, 379.5,  100,  100,  100,  200,   200,  200,  200,  200,  200]

# Percentages
per_load = 0.15 
per_rs   = 0.3
n_load   = 3

# dc
dc_u = list(df_dc[['U3', 'U4', 'U5', 'U6', 'U7', 'U8', 'Q0', 'Q1', 'Q2']].iloc[0])

### TA1

In [19]:
gp_opf_TA1 = GP_CCOPF(gp=gp, gp_method='TA1', predefine_probability_y=0.975, predefine_probability_u=0.999,
                      R=R, R2=R2, ulb=ulb, uub=uub, xlb=xlb, xub=xub, solver_opts=None, normalize=True,
                      X=X_train, Y=Y_train, Xd=Xd, alpha=alpha, per_load=per_load, per_rs=per_rs, n_load=n_load, A_coef=A_coef, b_inter=b_inter, dc_u=dc_u,
                      system='IEEE9')             

----------------------------------------
# Time to build GP CCOPF solver: 0.049395 sec


In [20]:
u_TA1, alpha_TA1, x_TA1, s_TA1, trace_covar_TA1, Xd_std = gp_opf_TA1.solve()

# Save outputs
gp_save = 0

if gp_save == 0:
    IEEE_dict = save_outputs.to_dict(x_TA1, s_TA1, u_TA1, alpha_TA1, trace_covar_TA1)
    save_outputs.save_model(IEEE_dict, 'outputs\IEEE9_TA1_hybrid')

Total number of variables............................:       36
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       32
Total number of inequality constraints...............:       54
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       18


Number of Iterations....: 6

                                   (scaled)                 (unscaled)
Objective...............:  1.4162666898336813e+003   2.4772422260764397e+003
Dual infeasibility......:  8.7862049139527536e-010   1.5368262189630473e-009
Constraint violation....:  5.6843418860808015e-014   5.6843418860808015e-014
Complementarity.........:  1.5874498149103428e-009   2.7766646928163939e-009
Overall NLP er

# Error metric 

In [21]:
gp = GPR.load_model('models/IEEE9_hybrid')


TA1_dict = save_outputs.load_model('outputs\IEEE9_TA1_hybrid')
x_TA1 = np.array(TA1_dict['x'])
s_TA1 = np.array(TA1_dict['s'])
u_TA1 = np.array(TA1_dict['u'])
alpha_TA1 = np.array(TA1_dict['s'])
u_TA1 = np.array(TA1_dict['trace_covar'])

In [24]:
df_pf_ac_mean = pd.read_csv('datasets\dataset_tested_real_AC.csv')
df_pf_ac_mean[['U3','U4','U5', 'U6', 'U7', 'U8']] = df_pf_ac_mean[['U3','U4','U5', 'U6', 'U7', 'U8']]*345

def _get_outputs_ac(df_ac):
    V_ac = df_ac[['U3', 'U4', 'U5', 'U6', 'U7', 'U8', 'Q0', 'Q1', 'Q2', 'P_34', 'P_45', 'P_56', 'P_67', 'P_78', 'P_83']]
    return V_ac 

out_pf_ac_mean = _get_outputs_ac(df_pf_ac_mean)

out_pf_ac_mean = np.array(out_pf_ac_mean)
out_pf_ac_meanout_pf_ac_mean = out_pf_ac_mean[:9, :]
out_pf_ac_mean.shape

(1, 15)

In [25]:
out_pf_mean_pu = np.zeros((1,15))
x_TA1_pu = np.zeros((1,15))
x_EM_pu = np.zeros((1,15))

out_pf_mean_pu[:,:6] = out_pf_ac_mean[:,:6]/345
out_pf_mean_pu[:,6:] = out_pf_ac_mean[:,6:]/100

x_TA1_pu[:,:6] = x_TA1[:,:6]/345
x_TA1_pu[:,6:] = x_TA1[:,6:]/100


RMSE_average = mean_squared_error(out_pf_mean_pu.T, x_TA1_pu.T, multioutput='uniform_average', squared=False)
RMSE_row = mean_squared_error(out_pf_mean_pu, x_TA1_pu, multioutput='raw_values', squared=False)

print('RMSE_avrage:', RMSE_average)
print('')
print('RMSE_row:', RMSE_row)

RMSE_avrage: 0.012245475084037356

RMSE_row: [0.00233309 0.00218584 0.00021289 0.00060605 0.00033521 0.00351028
 0.04067476 0.0060072  0.01979058 0.00314839 0.00355194 0.00298119
 0.00211155 0.00181374 0.01023974]
